In [74]:
from creating_arrays import CHART_TIME_ARRAY, MONTH_ARRAY, SYMBOL_ARRAY
from constants import BINANCE_URL, download_dir, output_dir, dtypes
from pathlib import Path
import os
import re
import shutil
from selenium import webdriver
import talib
import datetime
from selenium.webdriver.chrome.options import Options
import glob
import zipfile
import pandas as pd
from pandas.api.types import is_numeric_dtype
import inspect
import talib
import time
import numpy as np

In [5]:
# Get the current date
now = datetime.datetime.now()

# Format the date as a string in the desired format
date_string = now.strftime("%Y-%m")

# Print the date string
print(date_string)

idx = MONTH_ARRAY.index(date_string)
MONTH_ARRAY = MONTH_ARRAY[:idx + 1]

2023-01


In [9]:
# Set up the arrays of symbols, chart times, and months
symbols = [
    SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')],
#     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHUSDT')],
#     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHBUSD')],
#     SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCBUSD')]
]
# symbols = [SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')]]

# chart_times = CHART_TIME_ARRAY
chart_times = [CHART_TIME_ARRAY[12]]
months = MONTH_ARRAY

In [20]:
# # Iterate over the symbols and chart times
# for symbol in symbols:
#     for chart_time in chart_times:
#         # Construct the file name
#         file_name = f"{symbol}-{chart_time}.csv"

#         # Construct the file path
#         file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

#         # Load the data into a dataframe
#         df = pd.read_csv(file_path)
#         df = df[33:]
# # display(df[33:])

## LONGS


In [21]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]
#         df.dropna(inplace=True)
        # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances for symbol:", symbol, "chart_time",
              chart_time, ":", importances)

        # Get the indices of the top N features
        N = 2
        indices = np.argsort(importances)[::-1][:N]

        # Print the top N features
        print("Top {N} features for symbol:", symbol, "chart_time", chart_time,
              ":", [X.columns[i] for i in indices])

Feature importances for symbol: BTCUSDT chart_time 5m : [0.5179407 0.4820593]
Top {N} features for symbol: BTCUSDT chart_time 5m : ['RSI', 'MACD']


In [22]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]
        # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances for symbol:", symbol, "chart_time",
              chart_time, ":", importances)

        # Get the indices of the top N features
        N = 2
        indices = np.argsort(importances)[::-1][:N]

        # Print the top N features
        print("Top {N} features for symbol:", symbol, "chart_time", chart_time,
              ":", [X.columns[i] for i in indices])

#         # Get the range of RSI and MACD values that give the highest win rate
#         rsi_range = df["RSI"][(df["if_long"] == 1)].agg(["min", "max"])
#         macd_range = df["MACD"
        # Get the RSI and MACD values where if_long is 1 (long win)
        df_win = df[df["if_long"] == 1]
        rsi_win = df_win["RSI"]
        macd_win = df_win["MACD"]

        # Get the RSI and MACD values where if_long is 0 (long loss)
        df_loss = df[df["if_long"] == 0]
        rsi_loss = df_loss["RSI"]
        macd_loss = df_loss["MACD"]

        # Find the minimum and maximum values of RSI and MACD for long wins
        rsi_min_win = rsi_win.min()
        rsi_max_win = rsi_win.max()
        macd_min_win = macd_win.min()
        macd_max_win = macd_win.max()

        # Find the minimum and maximum values of RSI and MACD for long losses
        rsi_min_loss = rsi_loss.min()
        rsi_max_loss = rsi_loss.max()
        macd_min_loss = macd_loss.min()
        macd_max_loss = macd_loss.max()

        # Print the range of combination of RSI and MACD that gives the highest winrate of 71%
        print("Range of RSI for long wins:", rsi_min_win, "to", rsi_max_win)
        print("Range of MACD for long wins:", macd_min_win, "to", macd_max_win)
        print("Range of RSI for long losses:", rsi_min_loss, "to", rsi_max_loss)
        print("Range of MACD for long losses:", macd_min_loss, "to", macd_max_loss)


Feature importances for symbol: BTCUSDT chart_time 5m : [0.5179407 0.4820593]
Top {N} features for symbol: BTCUSDT chart_time 5m : ['RSI', 'MACD']
Range of RSI for long wins: 0.9916216347003368 to 98.69922666225663
Range of MACD for long wins: -1774.4048317051274 to 1272.1442664086353
Range of RSI for long losses: 21.475111831178783 to 64.62196028154327
Range of MACD for long losses: -15.811552105649753 to -2.6233224949210125


In [60]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]
        # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances for symbol:", symbol, "chart_time",
              chart_time, ":", importances)

        # Get the indices of the top N features
        N = 2
        indices = np.argsort(importances)[::-1][:N]

        # Print the top N features
        print("Top {N} features for symbol:", symbol, "chart_time", chart_time,
              ":", [X.columns[i] for i in indices])

        # Define the number of bins for RSI and MACD
        num_bins_rsi = 10
        num_bins_macd = 10

        # Get the predictions
        predictions = clf.predict(X)

        # Filter out the data points that were predicted to be winning trades
        X_win = X[predictions == 1]

        # Find the minimum and maximum values of RSI and MACD for winning trades
        rsi_min_win = X_win["RSI"].min()
        rsi_max_win = X_win["RSI"].max()
        macd_min_win = X_win["MACD"].min()
        macd_max_win = X_win["MACD"].max()

        # Print the range of RSI and MACD values that resulted in winning trades
        print("Range of RSI for winning trades:", rsi_min_win, "to", rsi_max_win)
        print("Range of MACD for winning trades:", macd_min_win, "to", macd_max_win)
        
        # Create bins for RSI and MACD values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), 20)
        macd_bins = np.linspace(df["MACD"].min(), df["MACD"].max(), 20)

        # Create new columns for RSI and MACD bins
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)
        df["MACD_bin"] = np.digitize(df["MACD"], macd_bins)

        # Get the number of wins in each RSI and MACD bin
        binned_data = df.groupby(["RSI_bin", "MACD_bin"])["if_long"].sum()

        # Find the RSI and MACD bin with the highest number of wins
        max_bin = binned_data.idxmax()

        # Get the RSI and MACD range for the bin with the highest number of wins
        rsi_range = (rsi_bins[max_bin[0]-1], rsi_bins[max_bin[0]])
        macd_range = (macd_bins[max_bin[1]-1], macd_bins[max_bin[1]])

        # Print the RSI and MACD range for the bin with the highest number of wins
        print("Range of RSI for winning trades:", rsi_range)
        print("Range of MACD for winning trades:", macd_range)



Feature importances for symbol: BTCUSDT chart_time 5m : [0.5179407 0.4820593]
Top {N} features for symbol: BTCUSDT chart_time 5m : ['RSI', 'MACD']
Range of RSI for winning trades: 0.9916216347003368 to 98.69922666225663
Range of MACD for winning trades: -1774.4048317051274 to 1272.1442664086353
Range of RSI for winning trades: (72.98669902342604, 78.12920455119216)
Range of MACD for winning trades: (10.831894193251856, 173.12614200219537)


In [65]:
# Get the data points where RSI and MACD are within the range and if_long is 0 (short trade)
df_range_short = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["MACD"] >= macd_range[0]) & (df["MACD"] < macd_range[1]) & (df["if_short"] == -1)]

# Get the total number of short trades within the range
total_short_trades = len(df_range_short)

# Calculate the win percentage for short trades within the range
win_percentage_short = df_range_short["if_long"].sum() / total_short_trades

# Print the results
print("Range of RSI:", rsi_range)
print("Range of MACD:", macd_range)
print("Total short trades:", total_short_trades)
print("Win percentage for short trades:", win_percentage_short)


Range of RSI: (72.98669902342604, 78.12920455119216)
Range of MACD: (10.831894193251856, 173.12614200219537)
Total short trades: 4373
Win percentage for short trades: 0.7018065401326321


In [35]:
len(df)

314433

## SHORTS


In [66]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:10000]
        df = df[df["if_short"] == 1]
       # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_short"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the predictions
        predictions = clf.predict(X)

        # Filter out the data points that were predicted to be short winning trades
        X_win = X[predictions == 1]

        # Find the minimum and maximum values of RSI and MACD for short winning trades
        rsi_min_win = X_win["RSI"].min()
        rsi_max_win = X_win["RSI"].max()
        macd_min_win = X_win["MACD"].min()
        macd_max_win = X_win["MACD"].max()

        # Print the range of RSI and MACD values that resulted in short winning trades
        print("Range of RSI for short winning trades:", rsi_min_win, "to", rsi_max_win)
        print("Range of MACD for short winning trades:", macd_min_win, "to", macd_max_win)


Range of RSI for short winning trades: 2.71561787892405 to 92.40092376838062
Range of MACD for short winning trades: -116.1173639839344 to 82.94444851213302


In [67]:
# Define the RSI and MACD range
rsi_range = (72.98669902342604, 78.12920455119216)
macd_range = (10.831894193251856, 173.12614200219537)

# Get the data points that fall within the RSI and MACD range
X_short = X[(df["RSI"] >= rsi_range[0]) & (X["RSI"] <= rsi_range[1]) & (df["MACD"] >= macd_range[0]) & (df["MACD"] <= macd_range[1])]

# Get the predictions for the data points that fall within the RSI and MACD range
predictions = clf.predict(X_short)

# Filter out the data points that were predicted to be losing trades
X_loss = X_short[predictions == 0]

# Get the total number of trades and win percentage
total_trades = len(X_loss)
win_percentage = (X_loss["if_short"].sum() / total_trades) * 100

# Print the total number of trades and win percentage
print("Total trades:", total_trades)
print("Win percentage:", win_percentage)


KeyError: 'if_short'

In [75]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Define the symbols and chart times
symbols = ["BTCUSDT"]
chart_times = ["5m"]


# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]
        # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_long"]
        
        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)
        
        # Define the number of bins for RSI and MACD
        num_bins_rsi = 20
        num_bins_macd = 20
        
        # Create bins for RSI and MACD values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)
        macd_bins = np.linspace(df["MACD"].min(), df["MACD"].max(), num_bins_macd)

        # Create new columns for RSI and MACD bins
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)
        df["MACD_bin"] = np.digitize(df["MACD"], macd_bins)
        
        # Get the number of long wins in each RSI and MACD bin
        win_data = df[df["if_long"] == 1].groupby(["RSI_bin", "MACD_bin"])["if_long"].count()
        loss_data = df[df["if_long"] == 0].groupby(["RSI_bin", "MACD_bin"])["if_long"].count()
        
        # Find the RSI and MACD bin with the highest number of wins
        max_win_bin = win_data.idxmax()
        max_loss_bin = loss_data.idxmax()
        
        # Get the RSI and MACD range for the bin with the highest number of wins
        rsi_range_win = (rsi_bins[max_win_bin[0]-1], rsi_bins[max_win_bin[0]])
        macd_range_win = (macd_bins[max_win_bin[1]-1], macd_bins[max_win_bin[1]])

        # Get the RSI and MACD range for the bin with the highest number of losses
        rsi_range_loss = (rsi_bins[max_loss_bin[0]-1], rsi_bins[max_loss_bin[0]])
        macd_range_loss = (macd_bins[max_loss_bin[1]-1], macd_bins[max_loss_bin[1]])

        # Print the RSI and MACD range for the bin with the highest number of wins
        print("Range of RSI for long wins:", rsi_range_win)
        print("Range of MACD for long wins:", macd_range_win)
        print("Range of RSI for long losses:", rsi_range_loss)
        print("Range of MACD for long losses:", macd_range_loss)

        # Filter the dataframe to only include trades within the RSI and MACD range for long wins
        df_win = df[(df["RSI"] >= rsi_range_win[0]) & (df["RSI"] < rsi_range_win[1]) & (df["MACD"] >= macd_range_win[0]) & (df["MACD"] < macd_range_win[1])]

        # Filter the dataframe to only include trades within the RSI and MACD range for long losses
        df_loss = df[(df["RSI"] >= rsi_range_loss[0]) & (df["RSI"] < rsi_range_loss[1]) & (df["MACD"] >= macd_range_loss[0]) & (df["MACD"] < macd_range_loss[1])]

        # Count the number of trades within the RSI and MACD range for long wins
        total_trades_win = df_win.shape[0]
        total_trades_loss = df_loss.shape[0]

        # Count the number of wins within the RSI and MACD range for long wins
        win_trades_win = df_win[df_win["if_long"] == 1].shape[0]
        win_trades_loss = df_loss[df_loss["if_long"] == 0].shape[0]

        # Calculate the win percentage within the RSI and MACD range for long wins
        win_percentage_win = win_trades_win / total_trades_win
        win_percentage_loss = win_trades_loss / total_trades_loss

        # Print the win percentage within the RSI and MACD range for long wins
        print("Win percentage within range for long wins:", win_percentage_win)
        print("Win percentage within range for long losses:", win_percentage_loss)


Range of RSI for long wins: (47.27417138459543, 52.41667691236155)
Range of MACD for long wins: (-151.46235361569188, 10.831894193251856)
Range of RSI for long losses: (47.27417138459543, 52.41667691236155)
Range of MACD for long losses: (-151.46235361569188, 10.831894193251856)
Win percentage within range for long wins: 0.4749169435215947
Win percentage within range for long losses: 9.966777408637874e-05


In [77]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]

        # Define the target variable
        y = df["if_long"]

        # Group the data by RSI and MACD
        grouped = df.groupby(["RSI", "MACD"])

        # Calculate the win rate for each group
        win_rate = grouped["if_long"].mean()

        # Filter the groups that have a win rate of 71% or higher
        win_rate = win_rate[win_rate >= 0.71]

        # Print the RSI, MACD, and win rate for the groups that have a win rate of 71% or higher
        display(win_rate)


RSI        MACD       
0.991622   -211.619657    1.0
1.118561   -153.885953    1.0
2.585793   -196.694637    1.0
3.138974   -84.375852     1.0
3.346156   -484.028685    1.0
                         ... 
97.450314   33.703307     1.0
97.601909   68.741575     1.0
97.630836   67.752398     1.0
98.551014   45.231726     1.0
98.699227   55.271351     1.0
Name: if_long, Length: 139960, dtype: float64

## WORKING CODE!!!!!!!!


In [79]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]


        # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances:", importances)

        # Define the number of bins for RSI and MACD
        num_bins_rsi = 10
        num_bins_macd = 10

        # Create bins for RSI and MACD values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)
        macd_bins = np.linspace(df["MACD"].min(), df["MACD"].max(), num_bins_macd)

        # Create new columns for RSI and MACD bins
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)
        df["MACD_bin"] = np.digitize(df["MACD"], macd_bins)

        # Get the number of wins in each RSI and MACD bin
        binned_data = df.groupby(["RSI_bin", "MACD_bin"])["if_long"].sum()

        # Find the RSI and MACD bin with the highest win rate
        win_rate = binned_data / df.groupby(["RSI_bin", "MACD_bin"]).size()
        max_win_rate_bin = win_rate[win_rate >= 0.71].idxmax()

        # Get the RSI and MACD range for the bin with the highest win rate
        rsi_range = (rsi_bins[max_win_rate_bin[0]-1], rsi_bins[max_win_rate_bin[0]])
        macd_range = (macd_bins[max_win_rate_bin[1]-1], macd_bins[max_win_rate_bin[1]])

        # Print the RSI and MACD range for the bin with the highest win rate
        print("Range of RSI for trades with win rate of 71% or higher:", rsi_range)
        print("Range of MACD for trades with win rate of 71% or higher:", macd_range)


Feature importances: [0.5179407 0.4820593]
Range of RSI for trades with win rate of 71% or higher: (11.848022193317703, 22.70442275193507)
Range of MACD for trades with win rate of 71% or higher: (-1774.4048317051274, -1431.7836418862466)


In [84]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:10000]


        # Define the features and target variables
        X = df[["RSI", "MACD"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances:", importances)

        # Define the number of bins for RSI and MACD
        num_bins_rsi = 10
        num_bins_macd = 10

        # Create bins for RSI and MACD values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)
        macd_bins = np.linspace(df["MACD"].min(), df["MACD"].max(), num_bins_macd)

        # Create new columns for RSI and MACD bins
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)
        df["MACD_bin"] = np.digitize(df["MACD"], macd_bins)

        # Get the number of wins and total trades in each RSI and MACD bin
        binned_data = df.groupby(["RSI_bin", "MACD_bin"])["if_long"].agg(["sum", "count"])

        # Add a new column for win percentage
        binned_data["win_percentage"] = binned_data["sum"] / binned_data["count"]

        # Find the bin or bins with win percentage greater than or equal to 0.71
        bins_with_high_win_percentage = binned_data[binned_data["win_percentage"] >= 0.71]

        # Print the range of RSI, MACD or a combination of both for the bin or bins with the highest win percentage
        for index, row in bins_with_high_win_percentage.iterrows():
            rsi_range = (rsi_bins[index[0]-1], rsi_bins[index[0]])
            macd_range = (macd_bins[index[1]-1], macd_bins[index[1]])
            print("Range of RSI:", rsi_range)
            print("Range of MACD:", macd_range)
            print("Win percentage:", row["win_percentage"])
            print()



Feature importances: [0.51091186 0.48908814]
Range of RSI: (2.71561787892405, 13.032804711207307)
Range of MACD: (-95.38144700171323, -73.0155624898507)
Win percentage: 1.0



IndexError: index 10 is out of bounds for axis 0 with size 10

In [88]:
target_win_rate = 0.71

# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]

        # Define the feature and target variable
        X = df[["RSI"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances for symbol:", symbol, "chart_time", chart_time, ":", importances)

        # Define the number of bins for RSI
        num_bins_rsi = 20

        # Create bins for RSI values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)

        # Create new column for RSI bin
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)

        # Get the number of wins in each RSI bin
        binned_data = df.groupby(["RSI_bin"])["if_long"].sum()

        # Find the RSI bin with the highest number of wins
        max_win_bin = binned_data.idxmax()

        # Get the RSI range for the bin with the highest number of wins
        rsi_range = (rsi_bins[max_win_bin-1], rsi_bins[max_win_bin])

        # Print the RSI range for the bin with the highest number of wins
        print("Range of RSI:", rsi_range)

        # Get the number of trades within the RSI range for long wins
        total_trades_win = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == 1)].shape[0]
        total_trades_loss = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == -1)].shape[0]

        # Get the number of trades within the RSI range for long wins
        total_trades_win = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == 1)].shape[0]
        total_trades_loss = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == -1)].shape[0]

        # Calculate the win percentage within the RSI range for long wins
        win_percentage_win = total_trades_win / (total_trades_win + total_trades_loss)
        win_percentage_loss = total_trades_loss / (total_trades_win + total_trades_loss)

        # Print the win percentage within the RSI range for long wins
        print("Win percentage for long wins:", win_percentage_win)
        print("Win percentage for long losses:", win_percentage_loss)
        
        if win_percentage_win >= target_win_rate or win_percentage_loss >= target_win_rate:
            print("RSI range with win rate greater than or equal to the target win rate:", rsi_range)
            break
        else:
            print("No RSI range with win rate greater than or equal to the target win rate found.")
            break


Feature importances for symbol: BTCUSDT chart_time 5m : [1.]
Range of RSI: (72.98669902342604, 78.12920455119216)
Win percentage for long wins: 0.6257270693512305
Win percentage for long losses: 0.3742729306487696
No RSI range with win rate greater than or equal to the target win rate found.


In [102]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]

        # Define the features and target variables
        X = df[["RSI"]]
        y = df["if_long"]

        # Create a random forest classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=0)

        # Fit the model to the data
        clf.fit(X, y)

        # Get the feature importances
        importances = clf.feature_importances_

        # Print the feature importances
        print("Feature importances for symbol:", symbol, "chart_time",
              chart_time, ":", importances)

        # Define the number of bins for RSI
        num_bins_rsi = 20

        # Create bins for RSI values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)

        # Create new column for RSI bin
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)

        # Get the number of wins in each RSI bin
        binned_data = df.groupby(["RSI_bin"])["if_long"].sum()

        # Get the total number of trades in each RSI bin
        binned_data_total = df.groupby(["RSI_bin"])["if_long"].count()

        # Calculate the win percentage in each RSI bin
        binned_data_win_percentage = binned_data / binned_data_total

        # Find RSI bin or bins with win percentage of 71% or higher
        bins_with_high_win_percentage = binned_data_win_percentage[binned_data_win_percentage >= 0]

        # Print the range of RSI and win percentage for each bin with win percentage of 71% or higher
        for rsi_bin, win_percentage in bins_with_high_win_percentage.items():
            rsi_range = (rsi_bins[rsi_bin-1], rsi_bins[rsi_bin])
#             rsi_range = (df["RSI"].min(), df["RSI"].max())
            total_trades = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1])].shape[0]
            total_wins = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == 1)].shape[0]
            print("Range of RSI:", rsi_range)
            print("Win percentage:", total_wins/total_trades)
            print("Total trades:", total_trades)
            print("Total wins:", total_wins)
            print()


Feature importances for symbol: BTCUSDT chart_time 5m : [1.]
Range of RSI: (62.70168796789379, 67.84419349565991)
Win percentage: 0.524099327547992
Total trades: 26619
Total wins: 13951

Range of RSI: (67.84419349565991, 72.98669902342604)
Win percentage: 0.5634418133468406
Total trades: 15794
Total wins: 8899

Range of RSI: (72.98669902342604, 78.12920455119216)
Win percentage: 0.6257270693512305
Total trades: 8940
Total wins: 5594

Range of RSI: (78.12920455119216, 83.27171007895828)
Win percentage: 0.6856357097524857
Total trades: 4727
Total wins: 3241

Range of RSI: (83.27171007895828, 88.4142156067244)
Win percentage: 0.758948799275034
Total trades: 2207
Total wins: 1675

Range of RSI: (88.4142156067244, 93.55672113449052)
Win percentage: 0.818734793187348
Total trades: 822
Total wins: 673

Range of RSI: (93.55672113449052, 98.69922666225663)
Win percentage: 0.8125
Total trades: 144
Total wins: 117



IndexError: index 20 is out of bounds for axis 0 with size 20

In [100]:
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)

        # Select the rows in the dataframe where RSI is between 83.27171007895828 and 88.4142156067244
        df = df[(df["RSI"] >= 83.27171007895828) & (df["RSI"] < 88.4142156067244)]

        # Calculate the number of wins in the selected RSI range
        total_wins = df[df["if_long"] == 1].shape[0]
        total_losses = df[df["if_long"] == -1].shape[0]

        # Calculate the total number of trades in the selected RSI range
        total_trades = df.shape[0]

        # Calculate the win percentage in the selected RSI range
        win_percentage = total_wins / total_trades

        # Print the results
        print("Symbol: ", symbol, "chart_time: ", chart_time)
        print("Win percentage:", win_percentage)
        print("Total trades:", total_trades)
        print("Total wins:", total_wins)
        print("Total losses:", total_losses)

Symbol:  BTCUSDT chart_time:  5m
Win percentage: 0.758948799275034
Total trades: 2207
Total wins: 1675
Total losses: 532
